### 2단계: Azure 서비스 구성

#### 2.1 Azure OpenAI Service (Private Endpoint)

**Azure Portal:**

1.  "Create a resource"에서 "Azure OpenAI"를 검색하여 선택하고 "Create"를 클릭합니다.
2.  다음 정보를 입력합니다.
    -   Subscription: 사용 중인 Azure 구독
    -   Resource group: "secu-ai-workshop"
    -   Region: "Korea Central"
    -   Name: "secu-ai-account"
    -   Pricing tier: Standard S0
    -   ![](images/20250459-155902.png)
3.  "Next: Network"를 클릭하고, "Connectivity type"을 "Disabled, no networks can access this resource."로 선택합니다.
    -   ![](images/20250404-160448.png)  
5.  "Add private endpoint"를 클릭하고 다음을 수행합니다.
    -   Name: "secu-ai-pe"
    -   Location: "Korea Central"
    -   Virtual Network: spoke1vnet
    -   Subnet: privateendpoint
    -   OK
    -   ![](images/20250406-160601.png)
6.  "Review + create"를 클릭하고 "Create"를 클릭합니다.
7.  Azure OpenAI Service가 배포된 후 gpt-4o 모델 배포
    -   배포된 secu-ai-account에서 Go to Azure AI Foundry portal을 클릭하여 이동
    -   Deployments에서 "Deploy model"에서 "Deploy base model"을 선택
    -   Model: gpt-4o 선택 후 버전 선택,
    -   Deployment name을 입력하고 Create.
    -   ![](images/20250416-161651.png)

#### 2.2 Private DNS zones for Azure OpenAI

**Azure Portal:**

1. "Private DNS zones"를 검색합니다.
2. 생성된 "privatelink.openai.azure.com"를 선택합니다.
3. "Virtual Network Links"에서 "+ Add"를 클릭합니다.
4. 다음을 입력합니다.
   -   Link name: "onprem-vnet-link"
   -   Virtual Network: onprem-vnet
   -   ![](images/20250444-114425.png)
5. "Create" 버튼을 클릭합니다.
6. 동일하게 hubvnet, spoke2vnet을 추가합니다.

In [1]:
# 기존 변수 설정
RESOURCE_GROUP="secu-ai-workshop"
LOCATION="koreacentral"

ONPREM_VNET_NAME="onprem-vnet"
ONPREM_ADDRESS_PREFIX="192.168.0.0/16"
ONPREM_SUBNET_NAME="onprem-subnet"
ONPREM_SUBNET_PREFIX="192.168.1.0/24"
GATEWAY_SUBNET_NAME="GatewaySubnet" # GatewaySubnet 이름은 고정
GATEWAY_SUBNET_PREFIX="192.168.0.0/24"

HUB_VNET_NAME="hubvnet"
HUB_ADDRESS_PREFIX="10.0.0.0/16"
HUB_GATEWAY_SUBNET_PREFIX="10.0.0.0/24"
FIREWALL_SUBNET_NAME="AzureFirewallSubnet" # AzureFirewallSubnet 이름은 고정
FIREWALL_SUBNET_PREFIX="10.0.1.0/26"
DMZ_SUBNET_NAME="dmz"
DMZ_SUBNET_PREFIX="10.0.2.0/24"

SPOKE1_VNET_NAME="spoke1vnet"
SPOKE1_ADDRESS_PREFIX="10.1.0.0/16"
SPOKE1_SUBNET_NAME="spoke1subnet"
SPOKE1_SUBNET_PREFIX="10.1.0.0/24"
SPOKE1_PE_SUBNET_NAME="privateendpoint"
SPOKE1_PE_SUBNET_PREFIX="10.1.1.0/24"

SPOKE2_VNET_NAME="spoke2vnet"
SPOKE2_ADDRESS_PREFIX="10.2.0.0/16"
SPOKE2_SUBNET_NAME="spoke2subnet"
SPOKE2_SUBNET_PREFIX="10.2.0.0/24"

In [4]:
# Azure OpenAI 관련 변수 설정
OPENAI_NAME="secu-ai-account1"
OPENAI_PE_NAME="secu-ai-pe"
OPENAI_PRIVATE_DNS_ZONE="privatelink.openai.azure.com"

In [ ]:
# Azure OpenAI account 생성
!az cognitiveservices account create \
    --name $OPENAI_NAME \
    --custom-domain $OPENAI_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --kind OpenAI \
    --sku s0

In [ ]:
# Azure OpenAI account의 Private Endpoint 생성
OPENAI_RESOURCE_ID=!az cognitiveservices account show --name $OPENAI_NAME --resource-group $RESOURCE_GROUP --query id -o tsv
OPENAI_RESOURCE_ID=OPENAI_RESOURCE_ID[0].strip()
!echo $OPENAI_RESOURCE_ID

In [ ]:

# Private Endpoint 생성
!az network private-endpoint create \
    --name $OPENAI_PE_NAME \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $SPOKE1_VNET_NAME \
    --subnet $SPOKE1_PE_SUBNET_NAME \
    --private-connection-resource-id $OPENAI_RESOURCE_ID \
    --group-ids account \
    --connection-name openai-connection \
    --location $LOCATION


In [ ]:
# Private DNS Zone 생성 (privatelink.openai.azure.com)
!az network private-dns zone create \
    --name $OPENAI_PRIVATE_DNS_ZONE \
    --resource-group $RESOURCE_GROUP

In [ ]:
# Private DNS Zone Link 생성 (Spoke1 VNET)
!az network private-dns link vnet create \
    --name spoke1vnetDnsLink \
    --resource-group $RESOURCE_GROUP \
    --zone-name $OPENAI_PRIVATE_DNS_ZONE \
    --virtual-network $SPOKE1_VNET_NAME \
    --registration-enabled false

In [ ]:
# Private DNS Zone Link 생성 (Onprem VNET)
!az network private-dns link vnet create \
    --name onpremvnetDnsLink \
    --resource-group $RESOURCE_GROUP \
    --zone-name $OPENAI_PRIVATE_DNS_ZONE \
    --virtual-network $ONPREM_VNET_NAME \
    --registration-enabled false

In [ ]:
# Private DNS Zone Link 생성 (Hub VNET)
!az network private-dns link vnet create \
    --name hubvnetDnsLink \
    --resource-group $RESOURCE_GROUP \
    --zone-name $OPENAI_PRIVATE_DNS_ZONE \
    --virtual-network $HUB_VNET_NAME \
    --registration-enabled false

In [ ]:
OPENAI_PRIVATE_DNS_ZONE_ID=!az network private-dns zone show --name $OPENAI_PRIVATE_DNS_ZONE --resource-group $RESOURCE_GROUP --query id -o tsv
OPENAI_PRIVATE_DNS_ZONE_ID=OPENAI_PRIVATE_DNS_ZONE_ID[0].strip()
!echo $OPENAI_PRIVATE_DNS_ZONE_ID

In [ ]:
# private dns record set create
!az network private-endpoint dns-zone-group create \
    -n "privatelink.openai.azure.com" \
    -g $RESOURCE_GROUP \
    --private-dns-zone $OPENAI_PRIVATE_DNS_ZONE \
    --endpoint-name $OPENAI_PE_NAME \
    --zone-name $OPENAI_PRIVATE_DNS_ZONE

In [ ]:
# Azure OpenAI account에 모델 배포. GPT-4o 모델을 사용합니다.
!az cognitiveservices account deployment create \
    --name $OPENAI_NAME \
    --resource-group $RESOURCE_GROUP \
    --deployment-name gpt-4o \
    --model-name gpt-4o \
    --model-version "2024-08-06" \
    --sku-capacity "100" \
    --sku-name "GlobalStandard" \
    --model-format "OpenAI"

#### 2.3 Azure API Management (Private Endpoint)

**Azure Portal:**

1.  "Create a resource"에서 "API Management"를 검색하여 선택하고 "Create"를 클릭합니다.
2.  다음 정보를 입력합니다.
    -   Subscription: 사용 중인 Azure 구독
    -   Resource group: "secu-ai-workshop"
    -   Region: "Korea Central"
    -   Name: "secu-ai-apim" (적당한 이름으로 변경)
    -   Organization name: "Contoso" (자신의 조직 이름)
    -   Administrator email: "admin@example.com" (자신의 이메일일)
    -   Pricing tier: Developer (테스트 용도)
    -   ![](images/20250427-162731.png)
    -   Connectivity type: "Private endpoint"
    -   ![](images/20250428-162840.png)
    -   Name: "apim-pe"
    -   Virtual Network: spoke1vnet
    -   subnet: privateendpoint
    -   ![](images/20250430-163034.png)
    -   System assigned managed identity : Check
    -   ![](images/20250431-163146.png)
3.  "Review + create" 를 클릭하고 "Create"를 클릭합니다.

#### 2.4 Private DNS zones for Azure API Management

**Azure Portal:**

1. "Private DNS zones"를 검색합니다.
2. 생성된 "privatelink.azure-api.net"를 선택합니다.
3. "Virtual Network Links"에서 "+ Add"를 클릭합니다.
4. 다음을 입력합니다.
   -   Link name: "onprem-vnet-link"
   -   Virtual Network: onprem-vnet
   -   ![](images/20250444-114425.png)
5. "Create" 버튼을 클릭합니다.
6. 동일하게 hubvnet, spoke2vnet을 추가합니다.

#### 2.5 Azure API Management public access disable

**Azure Portal:**

1. Cloud Shell 아이콘을 클릭합니다.
   -   ![](images/20250424145727.png)
2. Cloud Shell에서 다음 명령어를 입력합니다.
   -   az apim update --name (APIM NAME)  --resource-group (RESOURCE GROUP) --public-network-access false
   -   예시 : az apim update --name secu-ai-apim  --resource-group secu-ai-workshop --public-network-access false
   -   ![](images/20250424150037.png)

In [21]:
# 변수 설정
APIM_NAME="secu-ai-apim"
APIM_PE_NAME="apim-pe"
APIM_PRIVATE_DNS_ZONE="privatelink.azure-api.net"

In [19]:
# APIM 생성을 위한 VNET Subnet ID 가져오기
SPOKE1_VNET_ID=!az network vnet show --name $SPOKE1_VNET_NAME --resource-group $RESOURCE_GROUP --query id -o tsv
SPOKE1_VNET_ID=SPOKE1_VNET_ID[0].strip()
SPOKE1_PE_SUBNET_ID=!az network vnet subnet show --name $SPOKE1_PE_SUBNET_NAME --resource-group $RESOURCE_GROUP --vnet-name $SPOKE1_VNET_NAME --query id -o tsv
SPOKE1_PE_SUBNET_ID=SPOKE1_PE_SUBNET_ID[0].strip()


In [ ]:
# API Management 생성
!az apim create \
    --name $APIM_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --publisher-name "My Company" \
    --publisher-email "admin@mycompany.com" \
    --sku-name Developer

In [ ]:
# APIM에 Private Endpoint 생성을 위한 변수 설정정
APIM_RESOURCE_ID=!az apim show --name $APIM_NAME --resource-group $RESOURCE_GROUP --query id -o tsv
APIM_RESOURCE_ID=APIM_RESOURCE_ID[0].strip()
!echo $APIM_RESOURCE_ID

In [ ]:
# Private Endpoint 생성
!az network private-endpoint create \
    --name $APIM_PE_NAME \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $SPOKE1_VNET_NAME \
    --subnet $SPOKE1_PE_SUBNET_NAME \
    --private-connection-resource-id $APIM_RESOURCE_ID \
    --connection-name apim-connection \
    --location $LOCATION \
    --group-id Gateway 


In [ ]:
# APIM에 public endpoint을 사용하지 않도록 설정

!az apim update --name $APIM_NAME  --resource-group $RESOURCE_GROUP --public-network-access false --no-wait

In [ ]:
# Private DNS Zone 생성 (privatelink.azure-api.net)
!az network private-dns zone create \
    --name $APIM_PRIVATE_DNS_ZONE \
    --resource-group $RESOURCE_GROUP

In [ ]:
# Private DNS Zone Link 생성 (Spoke1 VNET)
!az network private-dns link vnet create \
    --name spoke1vnetApimDnsLink \
    --resource-group $RESOURCE_GROUP \
    --zone-name $APIM_PRIVATE_DNS_ZONE \
    --virtual-network $SPOKE1_VNET_NAME \
    --registration-enabled false

In [ ]:
# Private DNS Zone Link 생성 (Onprem VNET)
!az network private-dns link vnet create \
    --name onpremvnetApimDnsLink \
    --resource-group $RESOURCE_GROUP \
    --zone-name $APIM_PRIVATE_DNS_ZONE \
    --virtual-network $ONPREM_VNET_NAME \
    --registration-enabled false

In [ ]:
# Private DNS Zone Link 생성 (Hub VNET)
!az network private-dns link vnet create \
    --name hubvnetApimDnsLink \
    --resource-group $RESOURCE_GROUP \
    --zone-name $APIM_PRIVATE_DNS_ZONE \
    --virtual-network $HUB_VNET_NAME \
    --registration-enabled false

In [ ]:
# private dns record set create
!az network private-endpoint dns-zone-group create \
    -n "privatelink.azure-api.net" \
    -g $RESOURCE_GROUP \
    --private-dns-zone $APIM_PRIVATE_DNS_ZONE \
    --endpoint-name $APIM_PE_NAME \
    --zone-name $OPENAI_PRIVATE_DNS_ZONE

#### 2.6 Log Analytics (APIM 등 모니터링)

**Azure Portal:**

1.  "Create a resource"에서 "Log Analytics workspace"를 검색하여 선택하고 "Create"를 클릭합니다.
2.  다음 정보를 입력합니다.
    -   Subscription: 사용 중인 Azure 구독
    -   Resource group: "secu-ai-workshop"
    -   Name: "secu-ai-law"
    -   Region: "Korea Central"
    -   ![](images/20250434-163411.png)
3.  "Review + create"를 클릭하고 "Create"를 클릭합니다.

In [34]:
# 변수 설정
LOG_ANALYTICS_NAME="secu-ai-law"

In [ ]:
# Log Analytics Workspace 생성
!az monitor log-analytics workspace create \
    --name $LOG_ANALYTICS_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION

#### 2.7 APIM 진단 로그 설정

Azure Portal:

1.  생성된 API Management service인 "secu-ai-apim"에서 "monitoring"의 "Diagnostic settings"로 들어가서 "Add diagnostic setting"을 클릭합니다.
2.  다음 정보를 입력합니다.
    -   Diagnostic setting name: "apim-diagnostics"
    -   Logs와 Metrics의 모든 항목을 선택
    -   Send to Log Anaytics workspace 항목 선택
    -   Log Analytics workspace" "secu-ai-law" 선택
    -   ![](images/20250442-164227.png)
3.  "Save" 버튼을 눌러 설정을 저장합니다.

In [ ]:
# Log Analytics Workspace ID 가져오기
LOG_ANALYTICS_WORKSPACE_ID=!az monitor log-analytics workspace show --name $LOG_ANALYTICS_NAME --resource-group $RESOURCE_GROUP --query id -o tsv
LOG_ANALYTICS_WORKSPACE_ID=LOG_ANALYTICS_WORKSPACE_ID[0].strip()
!echo $LOG_ANALYTICS_WORKSPACE_ID


In [53]:
LOGS="[{category:GatewayLogs,enabled:true}]"
METRICS="[{category:AllMetrics,enabled:true}]"

In [ ]:
# APIM 진단 설정

!az monitor diagnostic-settings create \
    --name apim-diagnostics \
    --resource $APIM_NAME \
    --resource-group $RESOURCE_GROUP \
    --resource-type Microsoft.ApiManagement/service \
    --workspace $LOG_ANALYTICS_WORKSPACE_ID \
    --logs $LOGS \
    --metrics $METRICS

#### 2.8 APIM과 OpenAI Service 연결 및 Backend/API 설정 (CLI로 하지 않고 Azure Portal에서 설정정)

**APIM APIs 설정 (Azure Portal):**

1.  API Management 인스턴스 (secu-ai-apim)로 이동합니다.

2.  "APIs"를 클릭하고 "+ Add"를 클릭하고 "Azure OpenAI Service"를 선택합니다.
    -   ![](images/20250446-104637.png)

3.  다음 정보를 입력합니다.

    -   Azure OpenAI instance: secu-ai-account
    -   Display name: openaiapi
    -   Name: openaiapi
4.  Create

**APIM monitor 설정 (Azure Portal):**

1.  API Management 인스턴스 (secu-ai-apim)로 이동합니다.
2.  "APIs"를 클릭하고 생성된 "openaiapi"를 클릭합니다.
3.  "Setting" 탭에서 Azure Monitor 탭을 클릭합니다.
4.  "Override global"에 체크를 하고 "Number of payload bytes to log"에 "8192"를 입력합니다.
    -   ![](images/20250410-111052.png)
5.  "Save"버튼을 클릭합니다.


### 3단계: 테스트 환경 구성

#### 3.1 Virtual Machine (Onprem VNET에서 테스트 목적)

**Azure Portal:**

1.  "Create a resource"에서 "Virtual machine"을 검색하여 선택하고 "Create"를 클릭합니다.
2.  다음 정보를 입력합니다.
    -   Subscription: Azure 구독, Resource group: "secu-ai-workshop"
    -   Virtual machine name: "onprem-vm", Region: "Korea Central"
    -   Availability options: "No Infrastructure redundancy required"
    -   Security type: "Standard"
    -   Image: Ubuntu Server 24.04 LTS - x64 Gen2
    -   Size: B2as_v2 (또는 적절한 크기)
    -   ![](images/20250400-120039.png)
    -   Authentication type: Password
    -   Username: 관리자 계정 이름
    -   Password: 비밀 번호
    -   ![](images/20250413-121355.png)
    -   Networking: Virtual network: onprem-vnet, Subnet: onprem-subnet, Public IP: None
    -   ![](images/20250415-121506.png)
3.  "Review + create"를 클릭하고 "Create"를 클릭합니다.
4.  Bastion에서 Username과 VM Password를 입력하고 "Connect" 버튼을 눌러 VM의 콘솔에 들어갑니다.
    -   ![](images/20250450-125042.png)

In [64]:
# 변수 설정
VM_NAME="onprem-vm"
VM_IMAGE="Ubuntu2404"
VM_SIZE="Standard_B2s" # 필요에 따라 변경
VM_ADMIN_USERNAME="adminuser"
VM_ADMIN_PASSWORD="adminpassword1!" # 필요에 따라 변경

In [ ]:
# VM 생성 (패스워드 인증 방식)

!az vm create \
    --name $VM_NAME \
    --resource-group $RESOURCE_GROUP \
    --image $VM_IMAGE \
    --size $VM_SIZE \
    --vnet-name $ONPREM_VNET_NAME \
    --subnet $ONPREM_SUBNET_NAME \
    --admin-username $VM_ADMIN_USERNAME \
    --admin-password $VM_ADMIN_PASSWORD \
    --authentication-type password

In [ ]:
# Bastion Subnet 생성
!az network vnet subnet create \
    --name AzureBastionSubnet \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $ONPREM_VNET_NAME \
    --address-prefix 192.168.2.0/24

In [69]:
# Bastion Public IP 생성
BASTION_PIP_NAME="onprem-bastion-pip"
BASTION_NAME="onprem-bastion"

In [ ]:
# Bastion Public IP 생성
!az network public-ip create \
    --name $BASTION_PIP_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --sku Standard

In [ ]:
# Bastion 생성

!az network bastion create \
    --name $BASTION_NAME \
    --public-ip-address $BASTION_PIP_NAME \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $ONPREM_VNET_NAME \
    --location $LOCATION

### 3.2 OpenAI API 호출 테스트

**Azure Portal:**

1.  API Management의 Subscription Key를 복사합니다. 
    -   API Management에서 Subscription으로 이동한 후 Built-in all-access subscription의 ...버튼을 누르고 "Show/Hide Keys"를 누르고 Primary Key의 Subscription Key를 복사합니다.
    -   ![](images/20250424151740.png)
2.  Bastion을 통해 On-Prem VM에 접속 (Azure Portal)
    -   Virtual Machine 서비스로 이동하여 Onprem-vm을 선택합니다.
    -   해당 VM에서 Connect를 누르고 Connect via Bastion을 클릭합니다.
    -   ![](images/20250424153341.png)   
    -   VM의 user와 password를 입력하여 browser 에서 VM으로 들어갑니다.
3.  해당 VM에서 apim으로 api call을 합니다. 
    -   curl --request POST --url "https://(APIM NAME).azure-api.net/openaiapi/deployments/gpt-4o/chat/completions?api-version=2024-02-01" --header "api-key: (APIM Subscription Key)" --header "Content-Type: application/json" --data '{"messages":[{"role":"system","content":"You are a helpful assistant that provides detailed information about Microsoft Azure."},{"role":"user","content":"Azure VM 생성 절차를 알려줘"}]}'
    -   예시 : curl --request POST --url "https://secu-ai-apim.azure-api.net/openaiapi/deployments/gpt-4o/chat/completions?api-version=2024-02-01" --header "api-key: 3401e23463cc456db1ebe16479081969" --header "Content-Type: application/json" --data '{"messages":[{"role":"system","content":"You are a helpful assistant that provides detailed information about Microsoft Azure."},{"role":"user","content":"Azure VM 생성 절차를 알려줘"}]}'

### 3.4 Log Analytics에서 APIM을 통과한 OpenAI API 호출 결과 모니터링

**Azure Portal에서 Log Analytics 작업 영역(secu-ai-log)으로 이동**

1. Log Analytics Workspace에서 secu-ai-log를 선택
    -   "Logs" 블레이드 선택
    -   다음 쿼리들을 실행하여 APIM을 통해 들어온 OpenAI API 호출 결과 및 사용 토큰량 확인
    -   ![](images/20250424154733.png)
    -   ![](images/20250424154904.png) 

In [ ]:
ApiManagementGatewayLogs
| where tolower(OperationId) in ('completions_create','chatcompletions_create')
| where ResponseCode == '200'
| extend modelkey = substring(parse_json(BackendResponseBody)['model'], 0, indexof(parse_json(BackendResponseBody)['model'], '-', 0, -1, 2))
| extend model = tostring(parse_json(BackendResponseBody)['model'])
| extend prompttokens = parse_json(parse_json(BackendResponseBody)['usage'])['prompt_tokens']
| extend completiontokens = parse_json(parse_json(BackendResponseBody)['usage'])['completion_tokens']
| extend totaltokens = parse_json(parse_json(BackendResponseBody)['usage'])['total_tokens']
| extend ip = CallerIpAddress
| where model != ''
| summarize
    sum(todecimal(prompttokens)),
    sum(todecimal(completiontokens)),
    sum(todecimal(totaltokens)),
    avg(todecimal(totaltokens))
    by ip, model


In [ ]:
ApiManagementGatewayLogs
| where tolower(OperationId) in ('completions_create','chatcompletions_create')
| where ResponseCode  == '200'
| extend model = tostring(parse_json(BackendResponseBody)['model'])
| extend prompttokens = parse_json(parse_json(BackendResponseBody)['usage'])['prompt_tokens']
| extend prompttext = substring(parse_json(parse_json(BackendResponseBody)['choices'])[0], 0, 100)

**결과 확인**: curl 명령어가 정상 실행되면 OpenAI API로부터 응답(JSON)을 받음. 생성된 텍스트, 사용된 토큰 수 등 확인.

**이것으로 Azure Hub & Spoke 아키텍처를 활용하여 On-premise 환경에서 Azure OpenAI Service를 Private Endpoint를 통해 안전하게 호출하는 워크샵 구성을 완료했습니다.**